<img src="https://nlp.johnsnowlabs.com/assets/images/logo.png" width="180" height="50" style="float: left;">

# COLAB ENVIRONMENT SETUP

In [ ]:
# Fundamental Import and installation of Java
import os, shutil
from google.colab import drive
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Licensed Environment Setup
def setup_license_from_gdrive(mount_path, colab_path, aws_credentials_filename, license_filename):
    aws_dir = "~/.aws"
    if not os.path.exists(aws_dir):
        os.makedirs(aws_dir)
    shutil.copyfile(os.path.join(mount_path, colab_path, aws_credentials_filename),os.path.join(aws_dir, "credentials"))
    with open(os.path.join(mount_path, colab_path, license_filename), "r") as f:
        license = f.readline().replace("\n","")
        os.environ["JSL_NLP_LICENSE"] = license
    with open(os.path.join(mount_path, colab_path, secret_filename), "r") as f:
        secret = f.readline().replace("\n","")
    return secret

mount_path = '/content/gdrive'
colab_path = 'My Drive/Colab Notebooks'
aws_credentials_filename = 'credentials'
license_filename = 'license'
secret_filename = 'secret'
secret = setup_license_from_gdrive(mount_path, colab_path, aws_credentials_filename, license_filename)

# Intallation of Spark NLP Enterprise and its pyhon dependencies
! pip install spark-nlp-jsl==2.4.6 --extra-index-url https://pypi.johnsnowlabs.com/$secret

# Spark NLP Imports
from pyspark.sql import SparkSession
import sparknlp, sparknlp_jsl
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

# Creation of suitable SparkSession with proper JARS
spark = SparkSession.builder \
        .appName("Spark NLP Licensed") \
        .master("local[*]") \
        .config("spark.driver.memory", "8G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "1G") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.5") \
        .config("spark.jars", f"https://pypi.johnsnowlabs.com/{secret}/spark-nlp-jsl-2.4.6.jar")\
        .getOrCreate()

print("spark version:", spark.version)
print("spark-nlp version:", sparknlp.version())
print("spark-nlp-jsl version:", sparknlp_jsl.version())

# ICD-O Entity Resolution - version 2.4.6

## Example for ICD-O Entity Resolution Pipeline
A common NLP problem in medical applications is to identify histology behaviour in documented cancer studies.

In this example we will use Spark-NLP to identify and resolve histology behavior expressions and resolve them to an ICD-O code.

Some cancer related clinical notes (taken from https://www.cancernetwork.com/case-studies):  
https://www.cancernetwork.com/case-studies/large-scrotal-mass-multifocal-intra-abdominal-retroperitoneal-and-pelvic-metastases  
https://oncology.medicinematters.com/lymphoma/chronic-lymphocytic-leukemia/case-study-small-b-cell-lymphocytic-lymphoma-and-chronic-lymphoc/12133054
https://oncology.medicinematters.com/lymphoma/epidemiology/central-nervous-system-lymphoma/12124056
https://oncology.medicinematters.com/lymphoma/case-study-cutaneous-t-cell-lymphoma/12129416

Note 1: Desmoplastic small round cell tumor
<div style="border:2px solid #747474; background-color: #e3e3e3; margin: 5px; padding: 10px"> 
A 35-year-old African-American man was referred to our urology clinic by his primary care physician for consultation about a large left scrotal mass. The patient reported a 3-month history of left scrotal swelling that had progressively increased in size and was associated with mild left scrotal pain. He also had complaints of mild constipation, with hard stools every other day. He denied any urinary complaints. On physical examination, a hard paratesticular mass could be palpated in the left hemiscrotum extending into the left groin, separate from the left testicle, and measuring approximately 10 × 7 cm in size. A hard, lower abdominal mass in the suprapubic region could also be palpated in the midline. The patient was admitted urgently to the hospital for further evaluation with cross-sectional imaging and blood work.

Laboratory results, including results of a complete blood cell count with differential, liver function tests, coagulation panel, and basic chemistry panel, were unremarkable except for a serum creatinine level of 2.6 mg/dL. Typical markers for a testicular germ cell tumor were within normal limits: the beta–human chorionic gonadotropin level was less than 1 mIU/mL and the alpha fetoprotein level was less than 2.8 ng/mL. A CT scan of the chest, abdomen, and pelvis with intravenous contrast was obtained, and it showed large multifocal intra-abdominal, retroperitoneal, and pelvic masses (Figure 1). On cross-sectional imaging, a 7.8-cm para-aortic mass was visualized compressing the proximal portion of the left ureter, creating moderate left hydroureteronephrosis. Additionally, three separate pelvic masses were present in the retrovesical space, each measuring approximately 5 to 10 cm at their largest diameter; these displaced the bladder anteriorly and the rectum posteriorly.

The patient underwent ultrasound-guided needle biopsy of one of the pelvic masses on hospital day 3 for definitive diagnosis. Microscopic examination of the tissue by our pathologist revealed cellular islands with oval to elongated, irregular, and hyperchromatic nuclei; scant cytoplasm; and invading fibrous tissue—as well as three mitoses per high-powered field (Figure 2). Immunohistochemical staining demonstrated strong positivity for cytokeratin AE1/AE3, vimentin, and desmin. Further mutational analysis of the cells detected the presence of an EWS-WT1 fusion transcript consistent with a diagnosis of desmoplastic small round cell tumor.
</div>

Note 2: SLL and CLL
<div style="border:2px solid #747474; background-color: #e3e3e3; margin: 5px; padding: 10px"> 
A 72-year-old man with a history of diabetes mellitus, hypertension, and hypercholesterolemia self-palpated a left submandibular lump in 2012. Complete blood count (CBC) in his internist’s office showed solitary leukocytosis (white count 22) with predominant lymphocytes for which he was referred to a hematologist. Peripheral blood flow cytometry on 04/11/12 confirmed chronic lymphocytic leukemia (CLL)/small lymphocytic lymphoma (SLL): abnormal cell population comprising 63% of CD45 positive leukocytes, co-expressing CD5 and CD23 in CD19-positive B cells. CD38 was negative but other prognostic markers were not assessed at that time. The patient was observed regularly for the next 3 years and his white count trend was as follows: 22.8 (4/2012) --> 28.5 (07/2012) --> 32.2 (12/2012) --> 36.5 (02/2013) --> 42 (09/2013) --> 44.9 (01/2014) --> 75.8 (2/2015). His other counts stayed normal until early 2015 when he also developed anemia (hemoglobin [HGB] 10.9) although platelets remained normal at 215. He had been noticing enlargement of his cervical, submandibular, supraclavicular, and axillary lymphadenopathy for several months since 2014 and a positron emission tomography (PET)/computed tomography (CT) scan done in 12/2014 had shown extensive diffuse lymphadenopathy within the neck, chest, abdomen, and pelvis. Maximum standardized uptake value (SUV max) was similar to low baseline activity within the vasculature of the neck and chest. In the abdomen and pelvis, however, there was mild to moderately hypermetabolic adenopathy measuring up to SUV of 4. The largest right neck nodes measured up to 2.3 x 3 cm and left neck nodes measured up to 2.3 x 1.5 cm. His right axillary lymphadenopathy measured up to 5.5 x 2.6 cm and on the left measured up to 4.8 x 3.4 cm. Lymph nodes on the right abdomen and pelvis measured up to 6.7 cm and seemed to have some mass effect with compression on the urinary bladder without symptoms. He underwent a bone marrow biopsy on 02/03/15, which revealed hypercellular marrow (60%) with involvement by CLL (30%); flow cytometry showed CD38 and ZAP-70 positivity; fluorescence in situ hybridization (FISH) analysis showed 13q deletion/monosomy 13; IgVH was unmutated; karyotype was 46XY.
</div>

Note 3: CNS lymphoma
<div style="border:2px solid #747474; background-color: #e3e3e3; margin: 5px; padding: 10px"> 
A 56-year-old woman began to experience vertigo, headaches, and frequent falls. A computed tomography (CT) scan of the brain revealed the presence of a 1.6 x 1.6 x 2.1 cm mass involving the fourth ventricle (Figure 14.1). A gadolinium-enhanced magnetic resonance imaging (MRI) scan confirmed the presence of the mass, and a stereotactic biopsy was performed that demonstrated a primary central nervous system lymphoma (PCNSL) with a diffuse large B-cell histology. Complete blood count (CBC), lactate dehydrogenase (LDH), and beta-2-microglobulin were normal. Systemic staging with a positron emission tomography (PET)/CT scan and bone marrow biopsy showed no evidence of lymphomatous involvement outside the CNS. An eye exam and lumbar puncture showed no evidence of either ocular or leptomeningeal involvement.
</div>

Note 4: Cutaneous T-cell lymphoma
<div style="border:2px solid #747474; background-color: #e3e3e3; margin: 5px; padding: 10px"> 
An 83-year-old female presented with a progressing pruritic cutaneous rash that started 8 years ago. On clinical exam there were numerous coalescing, infiltrated, scaly, and partially crusted erythematous plaques distributed over her trunk and extremities and a large fungating ulcerated nodule on her right thigh covering 75% of her total body surface area (Figure 10.1). Lymphoma associated alopecia and a left axillary lymphadenopathy were also noted. For the past 3–4 months she reported fatigue, severe pruritus, night sweats, 20 pounds of weight loss, and loss of appetite. 
</div>

In [1]:
import sys, os, time, pandas as pd

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType

pd.set_option('display.max_colwidth', 250)
pd.set_option('display.max_rows', 500)

In [2]:
import sparknlp_jsl
sparknlp_jsl.version()

'2.4.6'

In [3]:
spark = sparknlp_jsl.start("####")

## Let's create a dataset with all four case studies

In [4]:
notes = []
notes.append("""A 35-year-old African-American man was referred to our urology clinic by his primary care physician for consultation about a large left scrotal mass. The patient reported a 3-month history of left scrotal swelling that had progressively increased in size and was associated with mild left scrotal pain. He also had complaints of mild constipation, with hard stools every other day. He denied any urinary complaints. On physical examination, a hard paratesticular mass could be palpated in the left hemiscrotum extending into the left groin, separate from the left testicle, and measuring approximately 10 × 7 cm in size. A hard, lower abdominal mass in the suprapubic region could also be palpated in the midline. The patient was admitted urgently to the hospital for further evaluation with cross-sectional imaging and blood work.
Laboratory results, including results of a complete blood cell count with differential, liver function tests, coagulation panel, and basic chemistry panel, were unremarkable except for a serum creatinine level of 2.6 mg/dL. Typical markers for a testicular germ cell tumor were within normal limits: the beta–human chorionic gonadotropin level was less than 1 mIU/mL and the alpha fetoprotein level was less than 2.8 ng/mL. A CT scan of the chest, abdomen, and pelvis with intravenous contrast was obtained, and it showed large multifocal intra-abdominal, retroperitoneal, and pelvic masses (Figure 1). On cross-sectional imaging, a 7.8-cm para-aortic mass was visualized compressing the proximal portion of the left ureter, creating moderate left hydroureteronephrosis. Additionally, three separate pelvic masses were present in the retrovesical space, each measuring approximately 5 to 10 cm at their largest diameter; these displaced the bladder anteriorly and the rectum posteriorly.
The patient underwent ultrasound-guided needle biopsy of one of the pelvic masses on hospital day 3 for definitive diagnosis. Microscopic examination of the tissue by our pathologist revealed cellular islands with oval to elongated, irregular, and hyperchromatic nuclei; scant cytoplasm; and invading fibrous tissue—as well as three mitoses per high-powered field (Figure 2). Immunohistochemical staining demonstrated strong positivity for cytokeratin AE1/AE3, vimentin, and desmin. Further mutational analysis of the cells detected the presence of an EWS-WT1 fusion transcript consistent with a diagnosis of desmoplastic small round cell tumor.""")
notes.append("""A 72-year-old man with a history of diabetes mellitus, hypertension, and hypercholesterolemia self-palpated a left submandibular lump in 2012. Complete blood count (CBC) in his internist’s office showed solitary leukocytosis (white count 22) with predominant lymphocytes for which he was referred to a hematologist. Peripheral blood flow cytometry on 04/11/12 confirmed chronic lymphocytic leukemia (CLL)/small lymphocytic lymphoma (SLL): abnormal cell population comprising 63% of CD45 positive leukocytes, co-expressing CD5 and CD23 in CD19-positive B cells. CD38 was negative but other prognostic markers were not assessed at that time. The patient was observed regularly for the next 3 years and his white count trend was as follows: 22.8 (4/2012) --> 28.5 (07/2012) --> 32.2 (12/2012) --> 36.5 (02/2013) --> 42 (09/2013) --> 44.9 (01/2014) --> 75.8 (2/2015). His other counts stayed normal until early 2015 when he also developed anemia (hemoglobin [HGB] 10.9) although platelets remained normal at 215. He had been noticing enlargement of his cervical, submandibular, supraclavicular, and axillary lymphadenopathy for several months since 2014 and a positron emission tomography (PET)/computed tomography (CT) scan done in 12/2014 had shown extensive diffuse lymphadenopathy within the neck, chest, abdomen, and pelvis. Maximum standardized uptake value (SUV max) was similar to low baseline activity within the vasculature of the neck and chest. In the abdomen and pelvis, however, there was mild to moderately hypermetabolic adenopathy measuring up to SUV of 4. The largest right neck nodes measured up to 2.3 x 3 cm and left neck nodes measured up to 2.3 x 1.5 cm. His right axillary lymphadenopathy measured up to 5.5 x 2.6 cm and on the left measured up to 4.8 x 3.4 cm. Lymph nodes on the right abdomen and pelvis measured up to 6.7 cm and seemed to have some mass effect with compression on the urinary bladder without symptoms. He underwent a bone marrow biopsy on 02/03/15, which revealed hypercellular marrow (60%) with involvement by CLL (30%); flow cytometry showed CD38 and ZAP-70 positivity; fluorescence in situ hybridization (FISH) analysis showed 13q deletion/monosomy 13; IgVH was unmutated; karyotype was 46XY.""")
notes.append("A 56-year-old woman began to experience vertigo, headaches, and frequent falls. A computed tomography (CT) scan of the brain revealed the presence of a 1.6 x 1.6 x 2.1 cm mass involving the fourth ventricle (Figure 14.1). A gadolinium-enhanced magnetic resonance imaging (MRI) scan confirmed the presence of the mass, and a stereotactic biopsy was performed that demonstrated a primary central nervous system lymphoma (PCNSL) with a diffuse large B-cell histology. Complete blood count (CBC), lactate dehydrogenase (LDH), and beta-2-microglobulin were normal. Systemic staging with a positron emission tomography (PET)/CT scan and bone marrow biopsy showed no evidence of lymphomatous involvement outside the CNS. An eye exam and lumbar puncture showed no evidence of either ocular or leptomeningeal involvement.") 
notes.append("An 83-year-old female presented with a progressing pruritic cutaneous rash that started 8 years ago. On clinical exam there were numerous coalescing, infiltrated, scaly, and partially crusted erythematous plaques distributed over her trunk and extremities and a large fungating ulcerated nodule on her right thigh covering 75% of her total body surface area (Figure 10.1). Lymphoma associated alopecia and a left axillary lymphadenopathy were also noted. For the past 3–4 months she reported fatigue, severe pruritus, night sweats, 20 pounds of weight loss, and loss of appetite.")

data = spark.createDataFrame([(i,n,) for i,n in enumerate(notes)], 
                             StructType([StructField("doc", StringType()),
                                         StructField("description", StringType())]))

## And let's build a SparkNLP pipeline with the following stages:
- DocumentAssembler: Entry annotator for our pipelines; it creates the data structure for the Annotation Framework
- SentenceDetector: Annotator to pragmatically separate complete sentences inside each document
- Tokenizer: Annotator to separate sentences in tokens (generally words)
- StopWordsCleaner: Annotator to remove words defined as StopWords in SparkML
- WordEmbeddings: Vectorization of word tokens, in this case using word embeddings trained from PubMed, ICD10 and other clinical resources.
- ChunkEmbeddings: Aggregates the WordEmbeddings for each NER Chunk
- BioNLP NER + NerConverter: This annotators return Chunks related to Cancer and Genetics diseases
- ChunkEntityResolver: Annotator that performs search for the KNNs, in this case trained from ICDO Histology Behavior.

In [5]:
#Usual preparation Annotators
docAssembler = DocumentAssembler().setInputCol("description").setOutputCol("document")
sentenceDetector = SentenceDetector().setInputCols("document").setOutputCol("sentence")
tokenizer = Tokenizer().setInputCols("sentence").setOutputCol("token")
stopCleaner = StopWordsCleaner().setOutputCol("clean_token")

#Embeddings and their aggregations per chunk
embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols("sentence", "clean_token")\
    .setOutputCol("embeddings")
chunkEmbeddings = ChunkEmbeddings()\
  .setInputCols("ner_chunk", "embeddings")\
  .setOutputCol("chunk_embeddings")

# Annotators responsible for the Canger Genetics Entity Recognition task
nerTagger = NerDLModel.pretrained("ner_bionlp", "en", "clinical/models")\
    .setInputCols("sentence", "clean_token","embeddings")\
    .setOutputCol("ner")
nerConverter = NerConverter().setInputCols("sentence","token","ner").setOutputCol("ner_chunk")\
.setWhiteList(['Cellular_component', 'Multi-tissue_structure', 'Organism_substance',
       'Gene_or_gene_product', 'Organism_subdivision', 'Cancer',
       'Cell'])

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_bionlp download started this may take some time.
Approximate size to download 13.9 MB
[OK!]


In [6]:
chunkResolverCM = ChunkEntityResolverModel.pretrained("chunkresolve_icd10cm_clinical", "en", "clinical/models")\
    .setNeighbours(100).setAlternatives(10)\
    .setEnableLevenshtein(True).setDistanceWeights([1,2,2,0,0,2])\
    .setInputCols("clean_token","chunk_embeddings")\
    .setOutputCol("icdcm_code")

chunkResolverO = ChunkEntityResolverModel.pretrained("chunkresolve_icdo_clinical", "en", "clinical/models")\
    .setNeighbours(100).setAlternatives(10)\
    .setEnableLevenshtein(True).setDistanceWeights([1,2,2,0,0,2])\
    .setInputCols("clean_token","chunk_embeddings")\
    .setOutputCol("icdo_code")

chunkresolve_icd10cm_clinical download started this may take some time.
Approximate size to download 166.3 MB
[OK!]
chunkresolve_icdo_clinical download started this may take some time.
Approximate size to download 8.2 MB
[OK!]


In [7]:
pipelineFull = Pipeline().setStages([
    docAssembler, 
    sentenceDetector, 
    tokenizer, 
    stopCleaner, 
    embeddings, 
    nerTagger,
    nerConverter,
    chunkEmbeddings, 
    chunkResolverCM,
    chunkResolverO])

Let's train our Pipeline and make it ready to start transforming

In [8]:
pipelineModelFull = pipelineFull.fit(data)

In [9]:
output = pipelineModelFull.transform(data).cache()

## The key parts of our model are the **WordEmbeddings and ChunkEntityResolver**: 

### WordEmebeddings:   
Word2Vec model trained on semantically augmented datasets using information from curated Datasets in JSL Data Market.  

### EntityResolver:  
Trained on an augmented ICDO Dataset from JSL Data Market it provides histology codes resolution for the matched expressions. Other than providing the code in the "result" field it provides more metadata about the matching process:  

- all_k_results -> Sorted ResolverLabels in the top `alternatives` that match the distance `threshold`
- all_k_resolutions -> Respective ResolverNormalized strings
- all_k_distances -> Respective distance values after aggregation
- all_k_wmd_distances -> Respective WMD distance values
- all_k_tfidf_distances -> Respective TFIDF Cosinge distance values
- all_k_jaccard_distances -> Respective Jaccard distance values
- all_k_sorensen_distances -> Respective SorensenDice distance values
- all_k_jaro_distances -> Respective JaroWinkler distance values
- all_k_levenshtein_distances -> Respective Levenshtein distance values
- all_k_confidences -> Respective normalized probabilities based in inverse distance values
- all_k_confidence_ratios -> Convenience indicator calculated as the ratio between the i_th and the (i+1)_th probability
- target_text -> The actual searched string
- resolved_text -> The top ResolverNormalized string
- confidence -> Top probability
- confidence_ratio -> Top confidence ratio
- distance -> Top distance value
- sentence -> Sentence index
- chunk -> Chunk Index
- token -> Token index

In [10]:
analysis_df = output\
.selectExpr("doc","""explode(arrays_zip(
ner_chunk.result,
ner_chunk.metadata,
icdcm_code.result,
icdo_code.result,
icdcm_code.metadata,
icdo_code.metadata
)) as arrays""")\
.where("float(arrays['4'].confidence_ratio) > 1.1")\
.selectExpr("doc",
            "arrays['0'] as chunk",
            "concat_ws('::',arrays['2'],float(arrays['4'].confidence_ratio)) as cmcode_conf",
            "split(arrays['4'].all_k_resolutions,':::') as cm_res",
            "concat_ws('::',arrays['3'],float(arrays['5'].confidence_ratio)) as ocode_conf",
            "split(arrays['5'].all_k_resolutions,':::') as o_res")\
.toPandas()

In [11]:
analysis_df

,doc,chunk,cmcode_conf,cm_res,ocode_conf,o_res
0,0,urinary,N3281::1.4048,"[Overactive bladder, Urinary calculus, unspecified, Unspecified urinary incontinence, Retention of urine, unspecified, Retention of urine, unspecified, Unspecified urinary incontinence, Poor urinary stream, Double urinary meatus, Urge incontinenc...",8312/3::1.0105,"[Renal cell carcinoma, Renal cell carcinoma, Renal cell carcinoma, Renal cell carcinoma, Renal cell carcinoma, Renal cell carcinoma, Renal cell carcinoma, Transitional cell carcinoma in situ, Transitional cell carcinoma in situ, Collecting duct c..."
1,0,blood cell,K920::1.3178,"[Hematemesis, Other blood donor, stem cells, Other blood donor, stem cells, Other decreased white blood cell count, Other elevated white blood cell count, Other elevated white blood cell count, Decreased white blood cell count, unspecified, Decre...",9732/3::1.0063,"[Multiple myeloma, Multiple myeloma, Plasma cell leukemia, Plasma cell leukemia, Plasma cell leukemia, Multiple myeloma, Multiple myeloma, Multiple myeloma, Plasma cell leukemia, Plasma cell leukemia]"
2,0,serum,E875::1.133,"[Hyperkalemia, Abnormal serum enzyme level, unspecified, Abnormal serum enzyme level, unspecified, Unspecified viral hepatitis B with hepatic coma, Unspecified viral hepatitis B without hepatic coma, Abnormal levels of other serum enzymes, Abnorm...",9533/0::1.0224,"[Psammomatous meningioma, Pineocytoma, Renal cell carcinoma, Plasma cell leukemia, Plasma cell leukemia, Plasma cell leukemia, Multiple myeloma, Multiple myeloma, Multiple myeloma, Multiple myeloma]"
3,0,less,R110::1.1445,"[Nausea, Burns involving less than 10% of body surface, Burns involving less than 10% of body surface, Burns involving less than 10% of body surface, Corrosions involving less than 10% of body surface, Corrosions involving less than 10% of body s...",8930/3::1.1843,"[Endometrial stromal sarcoma, Malignant mastocytosis, Mast cell sarcoma, Alveolar rhabdomyosarcoma, Psammomatous meningioma, Pineocytoma, Hairy cell leukemia, Oat cell carcinoma, Medulloblastoma, SHH-activated and TP53-mutant, Burkitt cell leukemia]"
4,0,alpha fetoprotein,E8801::1.1067,"[Alpha-1-antitrypsin deficiency, Alpha thalassemia, Alpha thalassemia, Alpha-1-antitrypsin deficiency, Alpha-1-antitrypsin deficiency, Immunoproliferative small intestinal disease, Pompe disease, Immunoproliferative small intestinal disease, Sucr...",9764/3::1.1434,"[Immunoproliferative small intestinal disease, Glucagonoma, malignant, Immunoproliferative small intestinal disease, Immunoproliferative small intestinal disease, Glucagonoma, malignant, Immunoproliferative small intestinal disease, Immunoprolife..."
5,0,pelvis,C763::1.1455,"[Malignant neoplasm of pelvis, Other osteonecrosis, pelvis, Other osteonecrosis, pelvis, Pathological fracture, pelvis, sequela, Stress fracture, pelvis, sequela, Solitary bone cyst, unspecified pelvis, Pathological fracture, pelvis, sequela, Str...",8312/3::1.0377,"[Renal cell carcinoma, Peripheral neuroectodermal tumor, Fibromyxosarcoma, Renal cell carcinoma, Renal cell carcinoma, Nephroblastoma, NOS, Renal cell carcinoma, sarcomatoid, Craniopharyngioma, Renal cell carcinoma, Adamantinomatous craniopharyng..."
6,0,left ureter,D3022::1.2748,"[Benign neoplasm of left ureter, Duplication of ureter, Megaloureter, Megaloureter, Agenesis of ureter, Duplication of ureter, Calculus of ureter, Hypermetropia, left eye, Tinnitus, left ear, Tinnitus, left ear]",8312/3::1.0673,"[Renal cell carcinoma, Craniopharyngioma, Nephroblastoma, NOS, Craniopharyngioma, Craniopharyngioma, Renal cell carcinoma, Adamantinomatous craniopharyngioma, Renal cell carcinoma, Renal cell carcinoma, Renal cell carcinoma]"
7,0,pelvic masses,M955::1.107,"[Acquired deformity of pelvis, Pelvic varices, Endometriosis of pelvic peritoneum, Postprocedural pelvic peritoneal adhesions, Postprocedural pelvic peritoneal adhesions, Unspecified parametritis and pelvic cellulitis, Unspecified parametritis an...",8312/3::1.099